<a href="https://colab.research.google.com/github/JoanPazUAB/BDnR_MongoDB/blob/main/lfsr_1606635.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Activity Pseudo-Random Generators
---
Name: Carla Martinez Vidal
Niu: 1606635

# Problem 1

---


1. We are going to implement now a simple LFSR as a Python class. We will use the notation and
structure of the LFSR described in the book (e.g. Figure 3.2). The LFSR is described with two
main parameters:
    1. Initial state: the value of the registers Sn, Sn−1, . . . , S2, S1. These values will be represented as
    a numpy array of ones and zeros. Be aware of the ordering!. For example the LFSR of 4 cells
    with initial state S4 = 1, S3 = 0, S2 = 1, S1 = 0 can be coded as: np.array([0, 1, 0, 1])
    2. Connection polynomial (polinomi de connexions): value of the ci parameters. So the values
    c1 = 0, c2 = 1, c3 = 0, c4 = 1 can be coded as: np.array([0, 1, 0, 1])

-The constructor receives the state and feedback polynomial and should initialize the current
state of the LFSR.

-The next() method computes one step of the LFSR returning one bit as output and updating
the current state of the LFSR.




In [ ]:
import numpy as np
import math
class LFSR():
    def __init__(self, state: np.ndarray, pol: np.ndarray):
        self.state = state
        self.pol = pol
        self.n = state.size

    def next(self) -> int:
        next_bit = self.state[-1]
        new_bit = 0
        for bit in range(self.n):
            if self.pol[bit] == 1:
                new_bit ^= self.state[bit]

        self.state = np.roll(self.state, 1)
        self.state[0] = new_bit
        return next_bit

In order to verify the code, your LFSR implementation should generate the following sequences when
using the following parameters:

In [ ]:
test_s1 = np.array([1,0,1])
test_c1 = np.array([1,1,1])
test_lfsr1 = LFSR(test_s1, test_c1)
test_seq1 = np.array([test_lfsr1.next() for _ in range(20)])
print(f"Test sequence 1:{test_seq1}")
test_s2 = np.array([1,0,0,0,1])
test_c2 = np.array([1,1,0,0,1])
test_lfsr2 = LFSR(test_s2, test_c2)
test_seq2 = np.array([test_lfsr2.next() for _ in range(20)])
print(f"Test sequence 2:{test_seq2}")


Test sequence 1:[1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0]
Test sequence 2:[1 0 0 0 1 0 1 1 0 0 0 1 0 1 1 0 0 0 1 0]


# Problem 2

---


2. With the LFSR we have implemented, generate the sequences of length 1000 bits, with the corresponding initial state si and initial values for ci:


In [ ]:
s1 = np.array([1,1,1,1,1,1])
c1 = np.array([1,0,0,0,0,1])
seclfsr1 = LFSR(s1, c1)
seq1 = np.array([seclfsr1.next() for _ in range(1000)])
#SECUENCIA 1
s2 = np.array([1,0,0,1,1,1])
c2 = np.array([1,0,0,1,1,0])
seclfsr2 = LFSR(s2, c2)
seq2 = np.array([seclfsr2.next() for _ in range(1000)])
#SECUENCIA 2
s3 = np.array([1,0,0,0,0,0])
c3 = np.array([0,0,0,0,1,0])
seclfsr3 = LFSR(s3, c3)
seq3 = np.array([seclfsr3.next() for _ in range(1000)])
#SECUENCIA 3

# Problem 3

---


3. Implement in Python the 3 tests from the NIST Test Suite for Random Number Generators (RNGs)
described in the book.


# TEST 1:

---


El test de freqüència de bits individuals comprova que la
proporció d’uns i zeros de la seqüència proporcionada és
similar.
Per fer-ho, en primer lloc es transforma la seqüència binària
d’entrada a una seqüència de 1 i -1:

In [ ]:

def rng_test_1(seq: np.ndarray) -> tuple[bool, float]:


    seq_1 = np.where(seq == 0, -1, 1)
    s_obs = np.sum(seq_1) / np.sqrt(seq_1.size)
    p_value = math.erfc(s_obs / math.sqrt(2))

    return p_value >= 0.01, p_value


In [ ]:
sequence = np.array([1,0,0,1,1,0,1,0,0,0,0,1,1,1,1,0,1,0,1,1,0,0,1])
b,f = rng_test_1(sequence)
print(b,f)

True 0.8348273291852392


# TEST 2:

---


El freqüència en un bloc comprova que el número de 0/1 en un bloc de m bits sigui
aproximadament m/2. Per fer-ho, es particiona la seqüència a avaluar en b = n / m
blocs de m bits, descartant els bits sobrants


In [ ]:
from scipy.special import gammainc
def rng_test_2(seq: np.ndarray, m: int = 10) -> tuple[bool, float]:

    b = seq.size // m
    x_obs = 0
    for i in range(0,seq.size,m):
        pi_k = np.sum(seq[i:i+m]) / m
        x_obs += ((pi_k) - 0.5 )**2

    x_obs = 4*m*x_obs
    p_value = float(gammainc(b / 2, x_obs / 2))
    return p_value >= 0.01, p_value


In [ ]:
sequence = np.array([1,0,0,1,1,0,1,0,0,0,0,1,1,1,1,0,1,0,1,1,0,0,1])
b,f = rng_test_2(sequence)
print(b,f)

True 0.9850044231795223


# TEST 3:

---


El test de ràfegues comprova si el número de ràfegues tant d’uns com de zeros de la
seqüència (N>100) s’assembla al que trobaríem en una seqüència aleatòria.
Definirem una ràfega com un conjunt de bits consecutius iguals, és a dir una ràfega de
longitud k consta dels elements

In [ ]:
def rng_test_3(seq: np.ndarray) -> tuple[bool, float]:

    v_obs = 1
    for i in range(seq.size-1):
        if seq[i] != seq[i+1]: v_obs +=1

    pi = sum(seq) / len(seq)
    numerator = abs(v_obs - (2.0 * len(seq) * pi * (1.0 - pi)))
    denominator = 2.0 * math.sqrt(2.0 * len(seq)) * pi * (1.0 - pi)
    value = math.erfc(numerator / denominator)
    return value >= 0.01, value


In [ ]:
sequence = np.array([1,0,0,1,1,0,1,0,0,0,0,1,1,1,1,0,1,0,1,1,0,0,1])
b,f = rng_test_3(sequence)
print(b,f)

True 0.5248996186322457


# Problem 4

---


4. For each sequence in question 2, run the three tests from question 3.

In [ ]:
secuencias = [seq1,seq2,seq3]
tests = [rng_test_1,rng_test_2,rng_test_3]
for i in range(len(secuencias)):
    for test in tests:
        print(f"test: {test.__name__} secuencia {i+1} resultado {test(secuencias[i])}")


test: rng_test_1 secuencia 1 resultado (True, 0.48661604576405054)
test: rng_test_2 secuencia 1 resultado (True, 0.08452672009769412)
test: rng_test_3 secuencia 1 resultado (True, 0.5169366858432001)
test: rng_test_1 secuencia 2 resultado (True, 0.9495709711511051)
test: rng_test_2 secuencia 2 resultado (False, 3.04279258146972e-100)
test: rng_test_3 secuencia 2 resultado (True, 0.999899074295685)
test: rng_test_1 secuencia 3 resultado (True, 2.0)
test: rng_test_2 secuencia 3 resultado (True, 1.0)
test: rng_test_3 secuencia 3 resultado (False, 1.7835329586941085e-15)


LINK COLAB: https://drive.google.com/file/d/1bU46K0-yTLMWgK__qhwhc01dam3_W5sK/view?usp=sharing
